In [1]:
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from PIL import Image
from tensorflow.python.keras import backend as K

In [2]:
tf.enable_eager_execution()
tf.executing_eagerly() 

True

In [3]:
x_train_filenames = glob.glob("data/images/*.jpeg")
x_train_filenames.sort()

y_train_filenames = glob.glob("data/masks/*.png")
y_train_filenames.sort()

In [4]:
x_train_filenames, x_val_filenames, y_train_filenames, y_val_filenames = \
                    train_test_split(x_train_filenames, y_train_filenames, test_size=0.2, random_state=42)

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train_filenames, y_train_filenames))
val_ds = tf.data.Dataset.from_tensor_slices((x_val_filenames, y_val_filenames))

In [6]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [7]:
def serialize_example(img,label_img):
    
    img_str = open(img,'rb').read()
    label_img_str =open(label_img,'rb').read()
    
    feature = {
        'image':_bytes_feature(img_str),
        'mask':_bytes_feature(label_img_str)
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [8]:
def tf_serialize_example(img, label_img):
    tf_string = tf.py_func(
        serialize_example, 
        (img, label_img), 
        tf.string)     
    return tf.reshape(tf_string, ()) # The result is a scalar

In [9]:
train_ds = train_ds.map(tf_serialize_example)
val_ds = val_ds.map(tf_serialize_example)

In [10]:
def save_tfrecord(filename, tfrecord):
    filename = filename+'.tfrecord'
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(tfrecord)

In [11]:
save_tfrecord("train_ds",train_ds)

In [12]:
save_tfrecord("val_ds",train_ds)